# Set up and import dataset

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

# Set path to local data directory
folder_path = "/Users/dais/Downloads/Optiver_additional data"

# Define file paths
feature_path = os.path.join(folder_path, "order_book_feature.parquet")
target_path = os.path.join(folder_path, "order_book_target.parquet")

# Load parquet files (first 30 mins = feature, last 30 mins = target side of the hour)
feature_df = pd.read_parquet(feature_path, engine='pyarrow')
target_df = pd.read_parquet(target_path, engine='pyarrow')

# Display dataset dimensions and column names
print("Feature DF:", feature_df.shape)
print("Target DF:", target_df.shape)
print("\nFeature columns:", feature_df.columns.tolist())
print("\nTarget columns:", target_df.columns.tolist())

# Concatenating both DataFrames vertically (stacking feature + target rows)
# Note: This doesn't align features and targets — it's just combining both halves of the hour
combined_df = pd.concat([feature_df, target_df], axis=0)

# Sort to organize by stock, time, and within-hour time buckets
combined_df = combined_df.sort_values(by=['stock_id', 'time_id', 'seconds_in_bucket']).reset_index(drop=True)

# Preview top and bottom of the stacked DataFrame
print("\nConcat DF (head):")
display(combined_df.head())

print("\nConcat DF (tail):")
display(combined_df.tail())


Feature DF: (17646119, 11)
Target DF: (17911332, 11)

Feature columns: ['stock_id', 'time_id', 'seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2']

Target columns: ['stock_id', 'time_id', 'seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2']

Concat DF (head):


,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,8382,6,1800.0,740.03,740.29,740.0,740.30,6,6,800,40
1,8382,6,1800.0,740.03,740.29,740.0,740.30,6,6,800,40
2,8382,6,1801.0,740.05,740.29,740.0,740.30,25,1,99,40
3,8382,6,1801.0,740.05,740.29,740.0,740.30,25,1,99,40
4,8382,6,1802.0,740.06,740.36,740.0,740.39,100,30,399,4



Concat DF (tail):


,stock_id,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
35557446,104919,1199,3595.0,362.73,362.74,362.72,362.75,200,1190,1200,1600
35557447,104919,1199,3596.0,362.68,362.70,362.67,362.71,800,200,1600,1400
35557448,104919,1199,3597.0,362.69,362.70,362.68,362.71,200,900,1400,1400
35557449,104919,1199,3598.0,362.72,362.73,362.71,362.74,200,1000,900,500
35557450,104919,1199,3599.0,362.80,362.81,362.79,362.82,200,300,600,300


# Feature Engineering

In [2]:
# Apply feature engineering function to the combined DataFrame (includes both first + last 30 min)
def compute_orderbook_features(df):
    """
    Compute engineered order book features from raw order book snapshots.
    """
    df = df.copy()

    # Mid price and Weighted Average Price
    df['mid_price'] = (df['bid_price1'] + df['ask_price1']) / 2
    df['wap'] = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (
        df['bid_size1'] + df['ask_size1']
    )

    # Spread and relative spread
    df['bid_ask_spread'] = df['ask_price1'] - df['bid_price1']
    df['spread_pct'] = df['bid_ask_spread'] / df['mid_price']

    # Spread variation over time within the same time_id
    df['spread_variation'] = df.groupby(['stock_id', 'time_id'])['spread_pct'].transform(
        lambda x: x.rolling(window=10, min_periods=1).std()
    )

    # Order book imbalance and depth ratio
    df['imbalance'] = (df['bid_size1'] - df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    df['depth_ratio'] = df['bid_size1'] / df['ask_size1'].replace(0, np.nan)

    # Return only the engineered features
    keep_cols = [
        'stock_id', 'time_id', 'seconds_in_bucket',
        'wap', 'spread_pct', 'imbalance', 'depth_ratio', 'spread_variation'
    ]
    return df[keep_cols]

# Apply feature engineering to the combined order book data
feature_engineered_df = compute_orderbook_features(combined_df)

# Preview result
feature_engineered_df.head()


,stock_id,time_id,seconds_in_bucket,wap,spread_pct,imbalance,depth_ratio,spread_variation
0,8382,6,1800.0,740.160000,0.000351,0.000000,1.000000,NaN
1,8382,6,1800.0,740.160000,0.000351,0.000000,1.000000,0.000000
2,8382,6,1801.0,740.280769,0.000324,0.923077,25.000000,0.000016
3,8382,6,1801.0,740.280769,0.000324,0.923077,25.000000,0.000016
4,8382,6,1802.0,740.290769,0.000405,0.538462,3.333333,0.000033


In [3]:
time_ref_path = os.path.join(folder_path, "time_id_reference.csv")
time_ref_df = pd.read_csv(time_ref_path)
time_ref_df.head()


,date,time,time_id
0,2021-01-05,11:00:00,12
1,2021-01-05,12:00:00,13
2,2021-01-05,13:00:00,14
3,2021-01-05,14:00:00,15
4,2021-01-05,15:00:00,16


# Combine date and time into a full datetime column


In [ ]:
time_ref_df["datetime"] = pd.to_datetime(time_ref_df["date"] + " " + time_ref_df["time"])
time_ref_df.head()

,date,time,time_id,datetime
0,2021-01-05,11:00:00,12,2021-01-05 11:00:00
1,2021-01-05,12:00:00,13,2021-01-05 12:00:00
2,2021-01-05,13:00:00,14,2021-01-05 13:00:00
3,2021-01-05,14:00:00,15,2021-01-05 14:00:00
4,2021-01-05,15:00:00,16,2021-01-05 15:00:00


: 

# Time ID to Real Time Mapping
NaT (Not a Time) means the merge didn’t find a matching time_id in time_ref_df
For example, time_id 6

In [ ]:
# Re-do the merge and keep only the new datetime column
feature_engineered_df = pd.merge(
    feature_engineered_df.drop(columns=["datetime"], errors="ignore"),  # just in case it already exists
    time_ref_df[["time_id", "datetime"]],
    on="time_id",
    how="left"
)
feature_engineered_df.head()
feature_engineered_df.tail()


# Train/Test Split 
## Chronological time_id-based split (80/20)

In [ ]:
# Sort just in case
feature_engineered_df = feature_engineered_df.sort_values(by="time_id")

# Unique time_ids
unique_ids = sorted(feature_engineered_df["time_id"].unique())
cutoff = int(len(unique_ids) * 0.8)

# Train on earliest 80%, test on latest 20%
train_ids = unique_ids[:cutoff]
test_ids = unique_ids[cutoff:]

train_df_timeid = feature_engineered_df[feature_engineered_df["time_id"].isin(train_ids)]
test_df_timeid = feature_engineered_df[feature_engineered_df["time_id"].isin(test_ids)]

print("TimeID Split:")
print("Train shape:", train_df_timeid.shape)
print("Test shape:", test_df_timeid.shape)


# Modeling with one stock 
Each stock has its own dynamics, so we usually train per stock

Each stock might behave diffrently to diffrent models and will required diffrent tuning 

In [ ]:
# Example with SPY XNAS
spy_df = feature_engineered_df[feature_engineered_df["stock_id"] == 50200].copy()
print(spy_df.tail())

# Check the datetime available
spy_df["date_only"] = spy_df["datetime"].dt.date




In [ ]:
spy_df.groupby("datetime")["spread_pct"].mean().plot(
    figsize=(15,4), title="SPY: Spread % Over Time"
)

In [ ]:

#  Compute log returns (per time_id)
spy_df["log_return"] = spy_df.groupby("time_id")["wap"].transform(lambda x: np.log(x / x.shift(1)))


# Compute realized volatility per time_id
rv_df = spy_df.groupby("time_id")["log_return"].agg(lambda x: np.sqrt(np.sum(x**2))).reset_index()

rv_df = rv_df.rename(columns={"log_return": "realized_volatility"})

#  Merge back into spy_df
spy_df = pd.merge(spy_df, rv_df, on="time_id", how="left")

# Plot volatility over time
spy_df.groupby("datetime")["realized_volatility"].mean().plot(
    figsize=(15, 4), title="SPY: Realized Volatility Over Time"
)
plt.xlabel("Datetime")
plt.ylabel("Realized Volatility")
plt.grid(True)
plt.show()



# HAV RV as baseline model
## train/test split on time id 80/20 with rolling window (W=330, H=10, S=5) 
## OLS

In [ ]:
# Helper function 

def evaluate_model(true, pred):
    # Clip predicted values to avoid log(0) or division by 0
    pred_clipped = np.clip(pred, 1e-8, None)
    true = np.array(true)
    
    # MSE
    mse = mean_squared_error(true, pred_clipped)

    # QLIKE
    try:
        qlike_score = np.mean(np.log(pred_clipped**2) + (true**2) / (pred_clipped**2))
    except Exception as e:
        qlike_score = np.nan
        print("QLIKE calculation failed:", e)

    print(f"Test MSE: {mse:.8f}")
    print(f"Test QLIKE: {qlike_score:.8f}")

    return mse, qlike_score


In [ ]:
# HAV RV OLS
# Prepare lagged HAV features
hav_df = rv_df.copy()
hav_df["rv_lag_1"] = hav_df["realized_volatility"].shift(1)
hav_df["rv_lag_5"] = hav_df["realized_volatility"].shift(5)
hav_df["rv_lag_10"] = hav_df["realized_volatility"].shift(10)
hav_df = hav_df.dropna().reset_index(drop=True)

# Set rolling window parameters
W, H, S = 330, 10, 5  # Window size, forecast horizon, step size

# Rolling window forecasting
all_preds = []
all_actuals = []

for start in range(0, len(hav_df) - W - H + 1, S):
    train_window = hav_df.iloc[start:start + W]
    test_window = hav_df.iloc[start + W:start + W + H]

    # Fit model on rolling window
    X_train = sm.add_constant(train_window[["rv_lag_1", "rv_lag_5", "rv_lag_10"]])
    y_train = train_window["realized_volatility"]
    model = sm.OLS(y_train, X_train).fit()

    # Predict
    X_test = sm.add_constant(test_window[["rv_lag_1", "rv_lag_5", "rv_lag_10"]])
    preds = model.predict(X_test)

    all_preds.extend(preds)
    all_actuals.extend(test_window["realized_volatility"].values)

# Evaluate 
preds = np.array(all_preds)
actuals = np.array(all_actuals)
#call helper function
mse_ols, qlike_ols = evaluate_model(actuals, preds)

# Plot predicted and actual volatility
plt.figure(figsize=(12, 4))
plt.plot(actuals, label="Actual RV", color="blue")
plt.plot(preds, label="Predicted RV (Rolling HAV-RV)", color="orange")
plt.title(f"HAV-RV with Rolling Window (W={W}, H={H}, S={S})\nMSE: {mse_ols:.6f}, QLIKE: {qlike_ols:.6f}")
plt.xlabel("Time ID (Rolling Forecast)")
plt.ylabel("Volatility")
plt.legend()
plt.grid(True)
plt.show()

## wls

In [ ]:
# HAV RV WLS
# Prepare lagged HAV features
hav_df = rv_df.copy()
hav_df["rv_lag_1"] = hav_df["realized_volatility"].shift(1)
hav_df["rv_lag_5"] = hav_df["realized_volatility"].shift(5)
hav_df["rv_lag_10"] = hav_df["realized_volatility"].shift(10)
hav_df = hav_df.dropna().reset_index(drop=True)

# Set rolling window parameters
W, H, S = 330, 10, 5  # Window size, forecast horizon, step size

# Rolling window forecasting
all_preds = []
all_actuals = []

for start in range(0, len(hav_df) - W - H + 1, S):
    train_window = hav_df.iloc[start:start + W]
    test_window = hav_df.iloc[start + W:start + W + H]

    # Compute weights (inverse of squared RV)
    weights = 1 / (train_window["realized_volatility"] ** 2 + 1e-8)  # add epsilon to avoid div by zero

    # Fit WLS model on rolling window
    X_train = sm.add_constant(train_window[["rv_lag_1", "rv_lag_5", "rv_lag_10"]])
    y_train = train_window["realized_volatility"]
    model = sm.WLS(y_train, X_train, weights=weights).fit()

    # Predict
    X_test = sm.add_constant(test_window[["rv_lag_1", "rv_lag_5", "rv_lag_10"]])
    preds = model.predict(X_test)

    all_preds.extend(preds)
    all_actuals.extend(test_window["realized_volatility"].values)

# Evaluate 
preds = np.array(all_preds)
actuals = np.array(all_actuals)
mse_wls, qlike_wls = evaluate_model(actuals, preds)

# Plot predicted and actual volatility
plt.figure(figsize=(12, 4))
plt.plot(actuals, label="Actual RV", color="blue")
plt.plot(preds, label="Predicted RV (Rolling HAV-WLS)", color="green")
plt.title(f"HAV-WLS with Rolling Window (W={W}, H={H}, S={S})\nMSE: {mse_wls:.6f}, QLIKE: {qlike_wls:.6f}")
plt.xlabel("Time ID (Rolling Forecast)")
plt.ylabel("Volatility")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
#!pip install statsmodels
#!pip install scikit-learn

